In [ ]:
"""
Input data
Adapted from https://github.com/architecture-building-systems/RC_BuildingSimulator/blob/master/simulator/BuildingProperties.py
"""

class box(object):
    def __init__(self, 
                 R_env = 42,   #resistance of envelope
                 Cm    =2.07   #capacitance of envelope
                ):
        #Single Capacitance Model Parameters
        self.Cm=Cm #[kWh/K] Room Capacitance. Default based of Madsen2011
        self.R_env=R_env #[K/kW] Wall resistance to outside air. Default based off glass having a Uvalue of 1.978W/m2K, 12m2 facade glass
        

In [56]:
"""
Import data
Adapted from https://github.com/architecture-building-systems/RC_BuildingSimulator/blob/master/simulator/input_data.py
"""

def read_EPW(epw_name='data/CH_ZH_original.epw'):
    #Should be done later with Pandas, but for some reason It's not working
    T_out=[] #Open empty matrix for storing dry bulb temperature values
    with open(epw_name, 'rb') as csvfile:
        weatherfile = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in weatherfile:
            if row[0].isdigit():
                T_out.append(row[6])
    return T_out

read_EPW

<function __main__.read_EPW>

In [26]:
def select_date_range(day,month,days):
    num_hours = 0
    start_hour = 0

    monthdays = [31,28,31,30,31,30,31,31,30,31,30,31]
    for m in range(0,int(month)-1):        
        start_hour+=monthdays[m]*24
        for d in range(0,int(day)-1):
            start_hour+=24

    end_hour = start_hour + days*24
    hour_list = range(start_hour, end_hour) #change this to range rather than list
    return hour_list

#select_date_range(1,2,2)

In [28]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)
fixed_df = pd.read_csv('data/CH_ZH_original.epw', sep=',')

fixed_df[int(select_date_range(1,2,2))+6]

TypeError: int() argument must be a string or a number, not 'list'

In [51]:
def temperature_range(day,month,days):
    T_list = []
    i=0

    for i in range(select_date_range(day,month,days)):
                   T_list.append(read_EPW[i])
    return i

temperature_range(1,1,3)

TypeError: range() integer end argument expected, got list.

In [ ]:
class weather
    def __init__(self, 
                 T_out,        #ambient temperature
                 T_in,         #indoor temperature at time m
                 del_t,        #discrete time step
                ):
        self.T_out = T_out
        self.T_in = T_in
        self.del_t = del_t

T_out: External temperature in C extracted from an EPW weather file

T_in: Calculated internal temperature

R_env: Resistance of the envelope. Must be caluclated by hand and inputted into the Building class in BuildingProperties.py

R_infl: Equivalent resistance due to infiltration. This is calculated within BuildingProperties.py

R_vent: Equivalent resistance due to ventelation. A variable resistance calculated by the setVentelation method in within the Building class of BuildingProperties.py

Cm: Capacitance of the room. Must be caluclated by hand and inputted into the Building class in BuildingProperties.py

Q_Heat: Heat energy supplied or removed by the heater or cooler. This is determined through a controller based on the temperature set points

Q_rad: Heat energy to the sun. Hourly radiation data through the windows must be determined in advance and read through the read_transmittedR funtion of input_data.py

Q_gains: Internal heat gains of people. Determined through the occupancy profile which is read in through the read_occupancy function of input_data.py

In [ ]:
"""
===========================
RC Model of a single zone building
===========================
File history and credits:
Justin Zarb
"""

import numpy as np
import math